In [1]:
from shapely.geometry import Point 
from shapely.geometry import LineString
import pandas as pd
import geopandas as gpd
import sys
import matplotlib.pyplot as plt
import datetime as datetime
import numpy as np
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

# Adding CSV Data

In [2]:
tornado_911 = pd.read_csv('../data/911_Phone_Calls_Tornado_030320.csv')
cadd_tornado = pd.read_csv('../data/Computer_Aided_Dispatch_Data_Tornado_Incidents_030320.csv', skiprows = 3)
derecho_911 = pd.read_csv('../data/911_Phone_Calls_Derecho_050320_050420.csv')
cadd_derecho = pd.read_csv('../data/Computer_Aided_Dispatch_Data_Derecho_Incidents_050320-050420.csv', skiprows = 3)

# Fixing Columns

In [3]:
tornado_911.columns = ['time', 'calltype', 'lat', 'lng', 'cell_tower_address']
derecho_911.columns = ['time', 'calltype', 'lat', 'lng', 'cell_tower_address']
cadd_tornado.columns = ['time', 'location', 'lat', 'lng', 'incident_type']
cadd_derecho.columns = ['time', 'location', 'lat', 'lng', 'incident_type']

In [4]:
tornado_911.time = pd.to_datetime(tornado_911.time)
derecho_911.time = pd.to_datetime(derecho_911.time)
cadd_tornado.time = pd.to_datetime(cadd_tornado.time)
cadd_derecho.time = pd.to_datetime(cadd_derecho.time)

In [5]:
tornado_911 = tornado_911.dropna(subset = ['lat', 'lng'])
derecho_911 = derecho_911.dropna(subset = ['lat', 'lng'])
cadd_tornado = cadd_tornado.dropna(subset = ['lat', 'lng'])
cadd_derecho = cadd_derecho.dropna(subset = ['lat', 'lng'])

# Adding geojson data

In [6]:
damage_points = gpd.read_file('../data/damage_points.geojson')
path_polygons = gpd.read_file('../data/path_polygons.geojson')
tornado_paths = gpd.read_file('../data/tornado_paths.geojson')

### Fixing columns

In [7]:
damage_points.stormdate = damage_points.stormdate.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))
damage_points.surveydate = damage_points.surveydate.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))
tornado_paths.stormdate = tornado_paths.stormdate.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))
tornado_paths.starttime = tornado_paths.starttime.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))
tornado_paths.endtime = tornado_paths.endtime.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))

# Filtering for the Nashville Tornado

In [8]:
nashville_tornado = tornado_paths.loc[tornado_paths.event_id == 'Nashville']
nashville_path = path_polygons.geometry.iloc[np.r_[4, 10:25, 26:35]]
nashville_path = nashville_path.reset_index()
nashville_path = nashville_path.drop(columns = ['index'])
nashville_damage = gpd.sjoin(damage_points, nashville_path, how = 'inner', op = 'within')
nashville_damage = nashville_damage.drop_duplicates(keep = 'first', subset = ['objectid'])

# Heatmaps

### Tornado

In [9]:
torn_list = []
for i in tornado_911['time'].unique():
    temp=[]
    for index, instance in tornado_911[tornado_911['time'] == i].iterrows():
        temp.append([instance['lat'], instance['lng']])
    torn_list.append(temp)

In [10]:
#creating a time index
time_index = []
for i in tornado_911['time'].unique():
    time_index.append(i)
#formatting the index
date_strings = [pd.to_datetime(d).strftime('%m/%d/%Y, %H:%M:%S') for d in time_index]

In [17]:
style = {'fillColor': 'red', 'color': 'red'}
style2 = {'fillColor': 'lightgreen', 'color': 'lightgreen'}
startloc = [36.20005, -86.41111]

torn_heat = folium.Map(location = startloc, zoom_start = 10)

HeatMapWithTime(torn_list,radius=25,auto_play=True,
                position='bottomright',name="cluster",index=date_strings,max_opacity=0.7).add_to(torn_heat)

folium.GeoJson(nashville_tornado.geometry, style_function = lambda x:style).add_to(torn_heat)

torn_heat.save('../maps/torn_heat.html')

torn_heat

### Derecho

In [12]:
derecho_list = []
for i in derecho_911['time'].unique():
    temp=[]
    for index, instance in derecho_911[derecho_911['time'] == i].iterrows():
        temp.append([instance['lat'], instance['lng']])
    derecho_list.append(temp)

In [13]:
#creating a time index
time_index1 = []
for i in derecho_911['time'].unique():
    time_index1.append(i)
#formatting the index
date_strings1 = [pd.to_datetime(d).strftime('%m/%d/%Y, %H:%M:%S') for d in time_index1]

In [14]:
startloc = [36.1627, -86.7816]

derecho_heat = folium.Map(location = startloc, zoom_start = 10)

HeatMapWithTime(derecho_list,radius=35,auto_play=True,
                position='bottomright', name="cluster", index=date_strings1, max_opacity=0.7).add_to(derecho_heat)

derecho_heat.save('../maps/derecho_heat.html')

derecho_heat

In [15]:
for index, damage_txt in enumerate(damage_points['damage_txt']):
    if damage_txt in ['Small Barns or Farm Outbuildings (SBO)', 'Metal Building System (MBS)', 
                     'Low-Rise Building [1-4 Stories] (LRB)', 'Electrical Transmission Lines (ETL)',
                     'Small Professional Building [Doctors Office, Branch Banks] (SPB)',
                     'Institutional Building [Hospital, Government or University Building] (IB)',
                     'Large, Isolated Retail Building [K-Mart, Wal-Mart] (LIRB)',
                     'Junior or Senior High School (JHSH)', 'Strip Mall (SM)',
                     'Warehouse Building [Tilt-up Walls or Heavy-Timber Construction] (WHB)',
                     'Elementary School [Single Story; Interior or Exterior Hallways] (ES)',
                     'Free-Standing Towers (FST)', 'Service Station Canopy (SSC)',
                     'Mid-Rise Building [5-20 Stories] (MRB)']:
        damage_points.loc[index, 'category'] = 'Commercial'
    elif damage_txt in ['One- or Two-Family Residences (FR12)', 'Masonry Apartment or Motel Building (MAM)',
                       'Manufactured Home - Single Wide (MHSW)', 'Apartments, Condos, Townhouses [3 stories or less] (ACT)',
                       'Manufactured Home - Double Wide (MHDW)', 'Motel (M)']:
        damage_points.loc[index, 'category'] = 'Residential'
    elif damage_txt in ['Trees: Hardwood (TH)', 'Trees: Softwood (TS)']:
        damage_points.loc[index, 'category'] = 'Trees'
    elif damage_txt in ['Other (O)']:
        damage_points.loc[index, 'category'] = 'Other'    